In [1]:
import sys 
import os
sys.path.append('/home/aistudio/external-libraries')

import numpy as np
import scipy as sp

import warnings
warnings.filterwarnings("ignore", category=Warning)

import paddle
import paddle.fluid as fluid
from paddle.static import InputSpec
from paddle.fluid.framework import core

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# from MulticoreTSNE import MulticoreTSNE as TSNE

import functools

import gc
from itertools import repeat
from collections import Counter
import tqdm
import json
import time

# from saliency.smooth_grad import SmoothGrad

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [2]:
# learning_rate = 1e-3
learning_rate = 2e-4
# learning_rate = 3e-1

batch_size = 128
# batch_size = 32

samples = 50000
test_samples = 10000
examples = samples
n_class = 100

training_epochs = 180

noise_rate = 0.8

image_size = 112

t_w = 1

In [3]:
class MyDataset(paddle.io.Dataset):
    def __init__(self, num_samples, file_path, save_path, transform, mode = 'train',):
        super(MyDataset, self).__init__()
        self.num_samples = num_samples
        self.transform = transform
        self.file_path = file_path

        self.dataset = paddle.vision.datasets.Cifar100(mode=mode, transform=self.transform, 
        download = False, data_file = file_path)

        if os.path.exists(save_path):
            self.labels = np.load(save_path)
        else:
            self.labels = []
            for d in tqdm.tqdm(self.dataset):
                self.labels.append(label_to_truelabel[str(d[1])])
            self.labels = np.array(self.labels).reshape(-1,)
            np.save(save_path, self.labels)
         

    def __getitem__(self, index):
        data = self.dataset[index][0]
        # data = np.array(data)
        
        # if self.transform is not None:
        #     data = self.transform(data)
        return index, data
    def __len__(self):
        return self.num_samples

In [4]:
import paddle.vision.transforms as T
from paddle.vision.transforms import functional as F
from paddle.vision.transforms import BaseTransform


mean,std = ([0.4914, 0.4822, 0.4465],[0.2471, 0.2435,0.2616])
mean = np.array(mean).reshape(1,1,3)*255
std = np.array(std).reshape(1,1,3)*255

transform_robust = T.Compose([
                    T.Resize(image_size),
                    T.Transpose(order = (2,1,0)),
                    T.RandomHorizontalFlip(0.5),
                    T.Normalize(mean=mean,std=std)
                    ])

transform_clean = T.Compose([
                    T.Resize(image_size),
                    T.Transpose(order=(2,1,0,)),
                    T.Normalize(mean=mean,std=std)
                    ])

transform_show = T.Compose([
                    T.Resize((image_size, image_size)),
                    ])
transform_new = T.Compose([
                    T.RandomHorizontalFlip(0.5),
                    T.RandomRotation(10),
                    T.Resize((image_size, image_size)),
                    T.Transpose(order=(2,1,0,)),
                    T.Normalize(mean=mean,std=std)
                    ])

print('Start download training data and load training data.')
train_dataset = MyDataset(samples, r'/home/aistudio/data/data76994/cifar-100-python.tar.gz',
r'./work/cifar-100-train_Y.npy',transform_robust, mode = 'train')

test_dataset = MyDataset(test_samples, r'/home/aistudio/data/data76994/cifar-100-python.tar.gz',
 r'./work/cifar-100-test_Y.npy',transform_clean, mode = 'test')

# d = paddle.vision.datasets.Cifar100(mode='train', transform=transform_robust, 
#         download = False, data_file = r'/home/aistudio/data/data76994/cifar-100-python.tar.gz')
print('Finished.')



Start download training data and load training data.
Finished.


In [5]:
train_loader = paddle.io.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=6,)
valid_loader = paddle.io.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=6,)


In [6]:

with open(r'work/label_to_truelabel.json') as f:
    label_to_truelabel = json.load(f)

with open(r'work/label_to_w.json') as f:
    label_to_w = json.load(f)
    
train_Y = np.load(r'./work/cifar-100-train_Y.npy').reshape(-1, )
test_Y = np.load(r'./work/cifar-100-test_Y.npy').reshape(-1, )
# try:
#     train_Y = np.load(r'./work/cifar-100-train_Y.npy').reshape(-1, )
#     test_Y = np.load(r'./work/cifar-100-test_Y.npy').reshape(-1, )
# except :
#     train_Y = []
#     for d in tqdm.tqdm(train_dataset):
#         train_Y.append(label_to_truelabel[str(d[1])])
#     train_Y = np.array(train_Y).reshape(-1,)
#     np.save(r'./work/cifar-100-train_Y.npy', train_Y)
#     test_Y = []
#     for d in tqdm.tqdm(test_dataset):
#         test_Y.append(label_to_truelabel[str(d[1])])
#     test_Y = np.array(test_Y).reshape(-1,)
#     np.save(r'./work/cifar-100-test_Y.npy', test_Y)


In [7]:
np.random.seed(1)
# noise_ = np.where(np.random.binomial(1, noise_rate, train_Y.shape), 1,0)
# train_Y_noised = train_Y.copy()
# for i in range(len(noise_)):
#     if noise_[i]>0:
#         if np.mod(train_Y_noised[i]+1,5) == 0:
#             train_Y_noised[i] -=4
#         else:
#             train_Y_noised[i] +=1
noise_=np.where(np.random.binomial(n=1,p=noise_rate,size=train_Y.shape),np.random.randint(low=1,high=n_class-1,size=train_Y.shape),0)

train_Y_noised = np.mod(noise_ + train_Y,n_class)
train_Y_noised = np.array(train_Y_noised)
Yt_list = [train_Y_noised]

In [8]:

class Orthogonal_loss17(paddle.nn.Layer):
    def __init__(self,):
        super(Orthogonal_loss17, self).__init__()
        
    def forward(self, x, ):
        n = x.shape[0]
        m = x.shape[1]

        I = paddle.eye(m)
        e = x - x.mean(axis=0, keepdim = True)
        
        cov = e.t() @ e
        
        cov2 = cov ** 2
        # cov2 = paddle.abs(cov)
        
        select_i = paddle.argmax(cov2 - cov2 * I, axis = 1)
        cov_m = (paddle.nn.functional.one_hot(select_i, m) * cov2).sum()
        cov_i = (I * cov).sum()
        
        result = (cov_m-cov_i) / (n*m)
        # result  = paddle.log(cov_m / cov_i + 1e-6)
        return result
        
loss_fn = paddle.nn.CrossEntropyLoss()

loss_ortho = Orthogonal_loss17()


In [9]:
probs_list = []
pred_list = []
Py_list = []
PP_list = []
Pm_list = []
acc_list = []
loss_list = []

warnings.filterwarnings("ignore", category=Warning) 

In [10]:

def cov_np(x, w=None):
    if w is None:
        n = x.shape[0]
        cov = x.T @ x / n
        e = np.mean(x, axis=0).reshape((-1, 1))
        res = cov - e @ e.T
    else:
        w = w.reshape((-1, 1))
        cov = (w * x).T @ x
        e = np.sum(w * x, axis=0).reshape((-1, 1))
        res = cov - e @ e.T

    res= res ** 2
    return res.sum() - np.trace(res)

def relevant_hard_np(x,):
    n = x.shape[0]
    nz = x.shape[1]
    e = x - x.mean(axis = 0,keepdims = True)

    cov = e.T @ e

    sigma = (e ** 2).sum(axis = 0, keepdims = True)
    r = cov / (sigma.T @ sigma) ** 0.5

    r = r ** 2
    r[np.isnan(r)] = 0.0

    return np.mean(np.max(r - r * np.eye(nz), axis = -1))

In [11]:
def validation():
    loss_eval = 0
    acc_eval = 0
    network_3.eval()
    PP_temp = np.zeros((test_samples,),dtype = np.float32)
    Py_temp = np.zeros((test_samples,),dtype = np.float32)
    Pm_temp = np.zeros((test_samples,),dtype = np.float32)
    Feature_l1_temp = np.zeros((test_samples,64),dtype = np.float32)
    Feature_l2_temp = np.zeros((test_samples,128),dtype = np.float32)
    Feature_l3_temp = np.zeros((test_samples,256),dtype = np.float32)
    Feature_temp = np.zeros((test_samples,network_3.fc.weight.shape[0]),dtype = np.float32)

    for batch_id, (ind, X_data) in enumerate(valid_loader()):
        ind = ind.numpy()
        Y_data = np.array(test_Y[ind]).astype(np.int64)
        temp_X = paddle.to_tensor(X_data)
        Y_GPU = paddle.to_tensor(Y_data)
        y_onehot = paddle.nn.functional.one_hot(paddle.reshape(Y_GPU,(-1,)),num_classes=n_class)

        first = network_3.maxpool(network_3.relu(network_3.bn1(network_3.conv1(temp_X))))
        l1 = network_3.layer1(first)
        l2 = network_3.layer2(l1)
        l3 = network_3.layer3(l2)
        l4 = network_3.layer4(l3)
        feature_l1 = paddle.reshape(network_3.avgpool(l1), (l1.shape[0],-1))
        feature_l2 = paddle.reshape(network_3.avgpool(l2), (l2.shape[0],-1))
        feature_l3 = paddle.reshape(network_3.avgpool(l3), (l3.shape[0],-1))
        feature = paddle.reshape(network_3.avgpool(l4), (l4.shape[0],-1))
                
        logits = network_3.fc(feature)
        probs = paddle.nn.functional.softmax(logits)
        
        PP = paddle.sum(probs * probs, axis = -1)
        Py = paddle.sum(y_onehot * probs, axis = -1)
        Pm = paddle.max(probs, axis = -1)
        PP_temp[ind] = PP.numpy()
        Py_temp[ind] = Py.numpy()
        Pm_temp[ind] = Pm.numpy()
        Feature_l1_temp[ind] = feature_l1.numpy()
        Feature_l2_temp[ind] = feature_l2.numpy()
        Feature_l3_temp[ind] = feature_l3.numpy()
        Feature_temp[ind] = feature.numpy()
        
        loss = loss_fn(logits, paddle.reshape(Y_GPU,(-1,1)))
        acc = paddle.metric.accuracy(logits, paddle.reshape(Y_GPU,(-1,1)))

        loss_eval += loss.numpy()
        acc_eval += acc.numpy()

    loss_eval/=(batch_id+1)
    acc_eval/=(batch_id+1)
    lossb = relevant_hard_np(np.concatenate([Feature_l1_temp, Feature_l2_temp, Feature_l3_temp, Feature_temp], axis = -1))
    # print('loss:%.4f, acc:%.4f, True_PP:%.4f, False_PP:%.4f'%(loss_eval, acc_eval, True_PP, False_PP))
    return loss_eval, acc_eval, lossb, Feature_l1_temp, Feature_l2_temp, Feature_l3_temp, Feature_temp

In [ ]:
network_3 = paddle.vision.models.resnet18(num_classes=n_class,pretrained=True)

scheduel = paddle.optimizer.lr.ExponentialDecay(learning_rate = learning_rate, gamma = 0.95)

opt = paddle.optimizer.Adam(learning_rate=learning_rate,
epsilon=1e-06,
parameters=network_3.parameters(),grad_clip = paddle.nn.ClipGradByGlobalNorm(clip_norm=1.0))
# opt = paddle.optimizer.SGD(learning_rate = scheduel,
#  parameters=network_3.parameters(),)


# logits_list = []
Pred_list = []
Py_list = []
PP_list = []
Pm_list = []
acc_list = []
loss_list = []

noise_n_list = []
select_n_list = []

lossb_list = []

score_list = [np.random.rand(samples,)]

softmax = paddle.nn.Softmax(axis=0)

start_time = time.time()

for epoch_id in range(training_epochs):
    network_3.train()
    feature_num = network_3.fc.parameters()[0].shape[0]
    loader = train_loader()

    loss_train = 0
    losso_train = 0
    acc_train = 0
    acc_train_ori = 0
    loss_train_ori = 0
    PP_temp = np.zeros((samples,),dtype=np.float32)
    Py_temp = np.zeros((samples,),dtype=np.float32)
    Pm_temp = np.zeros((samples,),dtype=np.float32)
    Pm_other_temp = np.zeros((samples,),dtype=np.float32)
    Pred_temp = np.zeros((samples,),dtype=np.float32)
    Probs_temp = np.zeros((samples,n_class),dtype=np.float32)
    Logits_temp = np.zeros((samples,n_class),dtype=np.float32)
    Logits_temp_other = np.zeros((samples,n_class),dtype=np.float32)

    feature_l1_temp = np.zeros((samples, 64), dtype = np.float32)
    feature_l2_temp = np.zeros((samples, 128), dtype = np.float32)
    feature_l3_temp = np.zeros((samples, 256), dtype = np.float32)
    feature_temp = np.zeros((samples, feature_num), dtype = np.float32)
 
    if len(Py_list)>0:
        PP_mean = PP_list[-1]
        Py_mean = Py_list[-1]
    else:
        PP_mean = np.zeros((samples,))
        Py_mean = np.zeros((samples,))

    thres = np.sort(score_list[-1])[int(samples*0.8)]
    r_mask = score_list[-1] >= thres
    side_mask = np.logical_and(r_mask,np.random.rand(samples,) < 0.2) 
    
    Y_onehot = np.eye(n_class)[Yt_list[-1]]
    Y_onehot_0 = np.eye(n_class)[Yt_list[0]]
    
    for batch_id, (ind, X_data) in enumerate(loader):
        ind = ind.numpy()

        Y_data = np.array(Yt_list[-1][ind]).astype(np.int64)
        Y_data_ori = np.array(train_Y[ind]).astype(np.int64)
        temp_X = paddle.to_tensor(X_data)
        Y_GPU = paddle.to_tensor(Y_data)
        Y_GPU_ori = paddle.to_tensor(Y_data_ori)
        y_onehot = paddle.nn.functional.one_hot(paddle.reshape(Y_GPU,(-1,)),num_classes=n_class)

        first = network_3.maxpool(network_3.relu(network_3.bn1(network_3.conv1(temp_X))))
        l1 = network_3.layer1(first)
        l2 = network_3.layer2(l1)
        l3 = network_3.layer3(l2)
        l4 = network_3.layer4(l3)
        feature_l1 = paddle.reshape(network_3.avgpool(l1), (l1.shape[0],-1))
        feature_l2 = paddle.reshape(network_3.avgpool(l2), (l2.shape[0],-1))
        feature_l3 = paddle.reshape(network_3.avgpool(l3), (l3.shape[0],-1))
        feature = paddle.reshape(network_3.avgpool(l4), (l4.shape[0],-1))

        logits = network_3.fc(feature)

        probs = paddle.nn.functional.softmax(logits)
        PP = paddle.sum(probs * probs, axis = -1)
        Py = paddle.sum(y_onehot * probs, axis = -1)
        Pm = paddle.max(probs, axis = -1)
        Pm_other = paddle.max(probs - probs * y_onehot, axis = -1)
        Pred = paddle.argmax(probs,axis=-1)
        logits_other = logits - y_onehot*1e10
        Pred_other = paddle.argmax(logits_other,axis=-1)

        alpha = 20
        # alpha = 1e-1

        if epoch_id < t_w:
            loss = loss_fn(logits,paddle.reshape(Y_GPU,(-1,1)))
            loss_o = paddle.to_tensor(np.array([0.]))
            loss_g = loss
        else:
            boundary_mask_ = paddle.to_tensor(side_mask[ind])
            label_rand = paddle.randint(low=0, high = n_class, shape=Y_GPU.shape)            
            Y_GPU = paddle.where(boundary_mask_, label_rand, Y_GPU)
            loss = loss_fn(logits,paddle.reshape(Y_GPU,(-1,1)))
            loss_o = loss_ortho(paddle.concat([feature_l1, feature_l2, feature_l3, feature], axis = -1))
            loss_g = loss + loss_o


            
        PP_temp[ind] = PP.numpy()
        Py_temp[ind] = Py.numpy()
        Pm_temp[ind] = Pm.numpy()
        Pm_other_temp[ind] = Pm_other.numpy()
        Pred_temp[ind] = Pred.numpy()

        Probs_temp[ind] = probs.numpy()
        Logits_temp[ind] = logits.numpy()
        Logits_temp_other[ind] = logits_other.numpy()
        feature_l1_temp[ind] = feature_l1.numpy()
        feature_l2_temp[ind] = feature_l2.numpy()
        feature_l3_temp[ind] = feature_l3.numpy()
        feature_temp[ind] = feature.numpy()
                
        acc = paddle.metric.accuracy(logits, paddle.reshape(Y_GPU,(-1,1)))
        acc_ori = paddle.metric.accuracy(logits, paddle.reshape(Y_GPU_ori,(-1,1)))
        loss_ori = loss_fn(logits,paddle.reshape(Y_GPU_ori,(-1,1)))

        loss_train += loss.numpy()
        losso_train += loss_o.numpy()
        acc_train += acc.numpy()
        acc_train_ori += acc_ori.numpy()
        loss_train_ori += loss_ori.numpy()
        
        loss_g.backward()
        
        opt.step()
        opt.clear_grad()
    scheduel.step()

        
    loss_train/=(batch_id+1)
    losso_train/=(batch_id+1)
    acc_train/=(batch_id+1)
    acc_train_ori/=(batch_id+1)
    loss_train_ori/=(batch_id+1)

    print('epoch %d train complete'%epoch_id)
    loss_eval, acc_eval, lossb_eval, feature_l1_val, feature_l2_val, feature_l3_val, feature_val = validation()

    loss_list.append(loss_eval)
    acc_list.append(acc_eval)
    Pred_list.append(Pred_temp)
    PP_list.append(PP_temp)
    Py_list.append(Py_temp)
    Pm_list.append(Pm_temp)
    
    lossb_list.append(lossb_eval)

    end_time = time.time()
    print(
        'train loss:%.4f,train losso:%.4f, train acc:%.4f, train acc ori:%.4f,\
          eval loss:%.4f, eval acc:%.4f, lossb eval:%e, time elapsed:%.4f'
    %(loss_train,losso_train, acc_train, acc_train_ori,
     loss_eval, acc_eval, lossb_eval, end_time - start_time))  
    
    if epoch_id < t_w:
        temp_Yt_noised = Yt_list[-1]
        select = np.zeros((samples,),dtype = np.bool)
        score = np.random.rand(samples,)
    else:
        feature_gpu = paddle.to_tensor(
            np.concatenate([feature_l1_temp, feature_l2_temp, feature_l3_temp, feature_temp, np.ones((samples,1))],
         axis = -1))
        Y_onehot_gpu = paddle.to_tensor(Y_onehot,dtype = paddle.float32)
        feature_gpu_val = paddle.to_tensor(
            np.concatenate([feature_l1_val, feature_l2_val, feature_l3_val, feature_val, np.ones((test_samples,1))],
         axis = -1))

        n_g = 5
        f_prob = np.zeros((samples, n_class))
        f_prob_val = np.zeros((test_samples, n_class))
        for i_w in range(n_g):
            m_v = paddle.to_tensor(np.where(np.random.rand(samples,) <= 0.8)[0])
            m_h = paddle.to_tensor(np.where(np.random.rand(feature_gpu.shape[1],) <= 0.5)[0])
            feature_gpu_mh = paddle.gather(feature_gpu, axis = 1, index = m_h)

            W = paddle.linalg.inv(feature_gpu_mh[m_v].T @ feature_gpu_mh[m_v]) @ feature_gpu_mh[m_v].T @ Y_onehot_gpu[m_v]

            f_prob = f_prob + (feature_gpu_mh @ W).numpy()
            f_prob_val = f_prob_val + (paddle.gather(feature_gpu_val, axis = 1, index = m_h) @ W).numpy()
        f_prob = f_prob / n_g
        f_prob_val = f_prob_val / n_g

        f_pred = np.argmax(f_prob, axis = -1)
        
        
        print('f_prob_l4_val : %.4f'%((np.argmax(f_prob_val, axis = -1) == test_Y).mean()))

        score = np.argsort(np.argsort(np.sum((Y_onehot_0 - f_prob)**2, axis = -1)))

        revise_predict = Pred_temp

        select = np.zeros((samples,),dtype= bool)


        r_ = min(0.1 + 0.1 * (epoch_id - t_w), 0.8)

        for j_ in range(n_class):
            class_mask = (Yt_list[0] == j_).ravel()
            class_n = class_mask.sum()
            if class_n > 0:
                class_thres = np.sort(score[class_mask])[int(class_n * (1 - r_))]
            else:
                class_thres = 1
            select[np.logical_and(class_mask, score >= class_thres)] = True
            
        temp_Yt_noised = np.where(select.ravel(), revise_predict.ravel(), Yt_list[0].ravel()).astype(int)

    is_noise = temp_Yt_noised != train_Y
    max_noised_class = -999
    for j_ in range(n_class):
        class_mask = train_Y == j_
        noise_n = np.logical_and(class_mask, is_noise).sum()
        if noise_n > max_noised_class:
            max_noised_class = noise_n
    Yt_list.append(temp_Yt_noised)
    print('epoch %d train cleaned, %d samples selected, %d samples changed'%(
        epoch_id,np.sum(Yt_list[-1]!=Yt_list[-2]), np.sum(Yt_list[-1]!=Yt_list[0])))
    Yt_remain_noise = np.sum(is_noise)
    print('total remain noise:%.4d, max class noise:%d'%(Yt_remain_noise, max_noised_class))
    noise_n_list.append(Yt_remain_noise)
    select_n_list.append(np.sum(select))
    score_list.append(score)


    # with open(r'./work_mechanism/cifar100/acc_eval_1226.txt','a') as f:
    #     f.write('%.8f\n'%acc_eval)
    # with open(r'./work_mechanism/cifar100/loss_eval_1226.txt','a') as f:
    #     f.write('%.8f\n'%loss_eval)
    # with open(r'./work_mechanism/cifar100/acc_train_ori_1226.txt','a') as f:
    #     f.write('%.8f\n'%acc_train_ori)
    # with open(r'./work_mechanism/cifar100/loss_train_ori_1226.txt','a') as f:
    #     f.write('%.8f\n'%loss_train_ori)
    # with open(r'./work_mechanism/cifar100/acc_train_1226.txt','a') as f:
    #     f.write('%.8f\n'%acc_train)
    # with open(r'./work_mechanism/cifar100/loss_train_1226.txt','a') as f:
    #     f.write('%.8f\n'%loss_train)

    # loss_unweighted = relevant_hard_np(feature_temp)
    
    # print('unweighted loss %e'%(loss_unweighted))
    gc.collect()

W0601 10:51:12.084864 17672 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0601 10:51:12.089440 17672 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


epoch 0 train complete
train loss:4.6811,train losso:0.0000, train acc:0.0358, train acc ori:0.1454,          eval loss:3.8370, eval acc:0.2565, lossb eval:3.373541e-01, time elapsed:22.2314
epoch 0 train cleaned, 0 samples selected, 0 samples changed
total remain noise:40030, max class noise:420
epoch 1 train complete
train loss:4.4714,train losso:0.0093, train acc:0.0740, train acc ori:0.3218,          eval loss:3.5773, eval acc:0.3679, lossb eval:2.343467e-01, time elapsed:43.8500
f_prob_l4_val : 0.3625
epoch 1 train cleaned, 5041 samples selected, 5041 samples changed
total remain noise:38199, max class noise:414
epoch 2 train complete
train loss:4.1667,train losso:0.0214, train acc:0.1398, train acc ori:0.4182,          eval loss:3.0928, eval acc:0.4149, lossb eval:2.131326e-01, time elapsed:66.5389
f_prob_l4_val : 0.3792
epoch 2 train cleaned, 10102 samples selected, 10039 samples changed
total remain noise:34692, max class noise:413
epoch 3 train complete
train loss:3.6984,train

In [ ]:
#cifar100, 80 uniform flip : 57.5
#cifar100, 60 uniform flip : 68.9
#cifar100, 40 uniform flip : 72.5
#cifar100, 20 uniform flip : 74.4

#cifar100, 40 pair flip : 72.8
#cifar100, 30 pair flip : 73.9
#cifar100, 20 pair flip : 75.1

In [ ]:
max(acc_list)

In [ ]:
# paddle.save(network_3.state_dict(), r'./work/network_3.pdparams')
# paddle.save(opt.state_dict(), r'./work/opt.pdopt')

In [ ]:
# np.save('./work/logits_list.npy',np.concatenate(logits_list, axis = 0))
# np.save('./work/logits_noise.npy',np.concatenate(logits_noise, axis = 0))